<a href="https://colab.research.google.com/github/gotica462/Player_Scouting/blob/main/resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import seaborn as sns
import os
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [2]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
events = pd.read_csv("events.csv")
events.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1.0,12.0,2.0,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0.0,9.0,2.0,1.0,1.0,0.0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2.0,NaN,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2.0,NaN,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3.0,NaN,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8.0,NaN,2.0,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0.0,2.0,NaN,0.0,NaN,0.0


In [ ]:
events.info()

In [6]:
#Show data columns

events.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [7]:
# Find out the values of the "is goal column"
events["is_goal"].unique()

array([ 0.,  1., nan])

In [8]:
#find out the count of "is goal golumn"
events["is_goal"].value_counts()

0.0    153979
1.0      3840
Name: is_goal, dtype: int64

In [9]:
# Reading the dictionary data to see what event type are 

dictionary_text = open('dictionary.txt')

In [10]:
dictionary_text.read()

'event_type\n0\tAnnouncement\n1\tAttempt\n2\tCorner\n3\tFoul\n4\tYellow card\n5\tSecond yellow card\n6\tRed card\n7\tSubstitution\n8\tFree kick won\n9\tOffside\n10\tHand ball\n11\tPenalty conceded\n\n\nevent_type2\n12\tKey Pass\n13\tFailed through ball\n14\tSending off\n15\tOwn goal\n\n\nside\n1\tHome\n2\tAway\n\n\nshot_place\n1\tBit too high\n2\tBlocked\n3\tBottom left corner\n4\tBottom right corner\n5\tCentre of the goal\n6\tHigh and wide\n7\tHits the bar\n8\tMisses to the left\n9\tMisses to the right\n10\tToo high\n11\tTop centre of the goal\n12\tTop left corner\n13\tTop right corner\n\n\nshot_outcome\n1\tOn target\n2\tOff target\n3\tBlocked\n4\tHit the bar\n\n\nlocation\n1\tAttacking half\n2\tDefensive half\n3\tCentre of the box\n4\tLeft wing\n5\tRight wing\n6\tDifficult angle and long range\n7\tDifficult angle on the left\n8\tDifficult angle on the right\n9\tLeft side of the box\n10\tLeft side of the six yard box\n11\tRight side of the box\n12\tRight side of the six yard box\n13\t

In [11]:
encoding = pd.read_table('dictionary.txt',  names=('num','events'))
encoding

,num,events
0,event_type,NaN
1,0,Announcement
2,1,Attempt
3,2,Corner
4,3,Foul
...,...,...
70,situation,NaN
71,1,Open play
72,2,Set piece
73,3,Corner


In [12]:
## checking the events column
events["event_type"].unique()

array([ 1.,  2.,  3.,  8., 10.,  7.,  9.,  4.,  6., 11., nan])

In [13]:
#find out the count of "event_type"
events["event_type"].value_counts()

8.0     41026
3.0     39959
1.0     36928
2.0     14507
7.0      8186
9.0      7841
4.0      6575
10.0     2125
11.0      448
6.0       224
Name: event_type, dtype: int64

In [14]:
# We need the is_goal  column for events and the event_type when is equal 1 since that is a shot attempt.
shot_attempts = events[events["event_type"]== 1]

# We check the filter
shot_attempts.head() 

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1.0,12.0,2.0,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0.0,9.0,2.0,1.0,1.0,0.0
11,UFot0hit/,UFot0hit12,12,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1.0,12.0,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,13.0,2.0,0.0,15.0,1.0,1.0,1.0,0.0
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1.0,12.0,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,4.0,1.0,1.0,9.0,2.0,1.0,1.0,0.0
14,UFot0hit/,UFot0hit15,15,19,Attempt blocked. Mats Hummels (Borussia Dortmu...,1.0,NaN,1.0,Borussia Dortmund,Hamburg SV,...,NaN,NaN,2.0,3.0,0.0,15.0,1.0,0.0,1.0,0.0
17,UFot0hit/,UFot0hit18,18,20,Attempt blocked. Tomas Rincon (Hamburg) right ...,1.0,NaN,2.0,Hamburg SV,Borussia Dortmund,...,NaN,NaN,2.0,3.0,0.0,15.0,1.0,0.0,1.0,0.0


In [15]:
### We'll check is the number is the same in both databases
shot_attempts["event_type"].value_counts()

1.0    36928
Name: event_type, dtype: int64

In [16]:
shot_attempts["player"].unique()

array(['mladen petric', 'shinji kagawa', 'kevin grosskreutz', ...,
       'sercan sararer', 'bernd nehrig', 'steed malbranque'], dtype=object)

In [17]:
# According to our dictionary database there are some shot locations that are not recorded (number 19)
print(shot_attempts.player[shot_attempts.location == 19].count()) 

201


In [18]:
# Let's see the percentage. 
print(shot_attempts.event_type[shot_attempts.location == 19].count()/shot_attempts["event_type"].value_counts()* 100) 

1.0    0.576798
Name: event_type, dtype: float64


In [19]:
#Since is less than 1% we are going to drop those values, even though it may affect our results, it will have a minimal effect

shot_attempts = shot_attempts[shot_attempts.location != 19]


In [20]:
#cHECK NULL VALUIES
shot_attempts.isna().any()

id_odsp          False
id_event         False
sort_order       False
time             False
text             False
event_type       False
event_type2       True
side             False
event_team       False
opponent         False
player           False
player2           True
player_in         True
player_out        True
shot_place        True
shot_outcome     False
is_goal          False
location         False
bodypart         False
assist_method    False
situation        False
fast_break       False
dtype: bool

In [21]:
#Let's start our training and testing
# Create our features
X = shot_attempts[['time', 'side', 'bodypart', 'location', 'situation', 'assist_method', 'fast_break']]

# Create our target
y = shot_attempts['is_goal']



In [25]:
#Let's start our training and testing
# Create our features
X = shot_attempts[['time', 'side', 'bodypart', 'location', 'situation', 'assist_method', 'fast_break']]

X = pd.get_dummies(X)

# Create our target
y = shot_attempts['is_goal']


In [26]:
X.head()

,time,side,bodypart,location,situation,assist_method,fast_break
0,2,2.0,2.0,9.0,1.0,1.0,0.0
11,14,1.0,1.0,15.0,1.0,1.0,0.0
13,17,1.0,2.0,9.0,1.0,1.0,0.0
14,19,1.0,1.0,15.0,1.0,0.0,0.0
17,20,2.0,1.0,15.0,1.0,0.0,0.0


In [27]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(27536, 7)

In [28]:
X.describe()

,time,side,bodypart,location,situation,assist_method,fast_break
count,36715.000000,36715.000000,36715.000000,36715.000000,36715.000000,36715.000000,36715.000000
mean,48.910037,1.438567,1.625385,10.268909,1.272695,1.137056,0.027700
std,26.342944,0.496218,0.740643,5.168233,0.696370,1.008845,0.164114
min,0.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000
25%,27.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000
50%,49.000000,1.000000,1.000000,11.000000,1.000000,1.000000,0.000000
75%,72.000000,2.000000,2.000000,15.000000,1.000000,2.000000,0.000000
max,90.000000,2.000000,3.000000,18.000000,4.000000,4.000000,1.000000


In [29]:
# Check the balance of our target values
y.value_counts()

0.0    33076
1.0     3639
Name: is_goal, dtype: int64

In [30]:
#Naive Random Oversampling
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0.0: 24807, 1.0: 24807})

In [31]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs', random_state=1)
logreg

# Fit 
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
# Calculate predictions
y_pred = logreg.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6115591930140243

In [33]:
# Calculate the confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[5198, 3071],
       [ 369,  541]])

In [34]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.93      0.63      0.59      0.75      0.61      0.37      8269
        1.0       0.15      0.59      0.63      0.24      0.61      0.37       910

avg / total       0.86      0.63      0.60      0.70      0.61      0.37      9179



In [37]:
#SMOTE Oversampling

# Resample the training data with SMOTEE

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0.0: 24807, 1.0: 24807})

In [38]:
# Train the Logistic Regression model using the resampled data

logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score

y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6112096151520507

In [40]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[5274, 2995],
       [ 378,  532]])

In [41]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.93      0.64      0.58      0.76      0.61      0.37      8269
        1.0       0.15      0.58      0.64      0.24      0.61      0.37       910

avg / total       0.86      0.63      0.59      0.71      0.61      0.37      9179



In [42]:
# Undersampling
# Resample the data using the ClusterCentroids resampler
#It's a large data so It will take some time.
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0.0: 2729, 1.0: 2729})

In [43]:
# Train the Logistic Regression model using the resampled data

logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [44]:
# Calculated the balanced accuracy score

y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.601443362539021

In [45]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[5585, 2684],
       [ 430,  480]])

In [46]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.93      0.68      0.53      0.78      0.60      0.36      8269
        1.0       0.15      0.53      0.68      0.24      0.60      0.35       910

avg / total       0.85      0.66      0.54      0.73      0.60      0.36      9179

